In [46]:
import sys
print(sys.path)

['D:\\Program Files\\JetBrains\\PyCharm 2022.1.4\\plugins\\python\\helpers-pro\\jupyter_debug', 'D:\\Program Files\\JetBrains\\PyCharm 2022.1.4\\plugins\\python\\helpers\\pydev', 'D:\\Hackathon-Summer-2023', 'D:\\Hackathon-Summer-2023', 'C:\\Users\\jibi\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip', 'C:\\Users\\jibi\\AppData\\Local\\Programs\\Python\\Python310\\DLLs', 'C:\\Users\\jibi\\AppData\\Local\\Programs\\Python\\Python310\\lib', 'C:\\Users\\jibi\\AppData\\Local\\Programs\\Python\\Python310', 'D:\\JIBI\\pycharm笔记相关\\venv', '', 'D:\\JIBI\\pycharm笔记相关\\venv\\lib\\site-packages', 'D:\\JIBI\\pycharm笔记相关\\venv\\lib\\site-packages\\win32', 'D:\\JIBI\\pycharm笔记相关\\venv\\lib\\site-packages\\win32\\lib', 'D:\\JIBI\\pycharm笔记相关\\venv\\lib\\site-packages\\Pythonwin']


In [47]:
from IPython.display import display
import anndata
import pandas as pd
import numpy as np
import my_fuc as mf
import importlib
importlib.reload(mf)
import scanpy as sc
print('scanpy_test')

scanpy_test


In [48]:
test_path = 'test_data/sce.h5ad'
test_data = anndata.read_h5ad(test_path)#等价于scanpy.read_h5ad(..)
display(test_data)
display(type(test_data))

AnnData object with n_obs × n_vars = 21954 × 25269
    uns: 'X_name'

anndata._core.anndata.AnnData

In [49]:
print(test_data.obs)#查看观察元数据
print(type(test_data.obs))#dataframe,空列，只有行索引有内容

print(test_data.var)# 查看变量元数据
print(type(test_data.var))

print(test_data.X)# 查看数据拒阵，但只提供非零教据相当于是非0数据的索引表内容:(16151，25267) 1,前者为元组，16151行的25267列的内容非零(为1)
print(type(test_data.X))#<class 'scipy.sparse._csc.csc_matrix'>
print(test_data.X.shape)#(21954, 25269)原形状
# print(test_data.uns)# uns属性是未排序的元数据字典，其中可能包含与数据相关的其他信息{'X_name': 'counts'}
# print(test_data.uns['X_name'])#uns属性中X_name键对应的值为counts  counts

Empty DataFrame
Columns: []
Index: [AGAGCGAAGATCTGCT, ACATCAGTCTGACCTC, TTCTCCTGTCCTGCTT, GAAACTCTCATGTAGC, CAGTCCTGTCTAGCCG, CATGCCTCACACCGCA, AGGCCGTAGGCCCGTT, AAGTCTGAGCGTCTAT, CAGTAACCAGGAACGT, GTCTCGTTCGTACCGG, CCACGGAAGCACACAG, CGCTATCAGTGAAGTT, CGGAGCTCACATTTCT, AGATTGCAGCTGATAA, AGATTGCGTTTGGGCC, AGCGTATTCGACCAGC, GTGTTAGTCGTCGTTC, ACGGGCTCAGGTCTCG, TACTCATAGTACGTTC, CAACCTCGTACGCACC, CGATGTATCTCACATT, GATCGTAGTAAATGTG, CATGACATCTATCGCC, CATATGGCAACTGCGC, ACCGTAAGTCTAGCGC, TGGTTCCAGTCCTCCT, AGCAGCCAGTGCCATT, GGCGTGTTCAGGCAAG, CACAAACAGTGCGATG, ATGGGAGCAGATCGGA, ATCACGATCACTATTC, CTTCTCTAGCCCTAAT, AACTGGTTCGGTTCGG, GACAGAGAGCCCGAAA, ACGATGTAGCGATGAC, CCGTGGACATCTACGA, GGAACTTCATAGGATA, TCGGGACCAAAGAATC, TACACGATCGCTTAGA, ATGTGTGTCCAAAGTC, GCTGCGATCTTCGGTC, GCAATCACAGCTCGAC, GCAGTTACACATGGGA, ATCATCTTCTGCTGCT, GCTGGGTAGACTCGGA, TTCTACACACCGTTGG, CTCGTACCAATGGAGC, CGAGAAGCATGCGCAC, TGAGGGACAGCCTGTG, GGACAGATCCCAAGAT, CATTCGCGTACCAGTT, GTTTCTAAGTCGATAA, CCCATACTCGGTGTTA, ACTTTCAAGA

获得原始矩阵

In [50]:
%%time
dense_matrix = test_data.X.toarray()#非零稀疏索引转为二维数组
display(dense_matrix)
display(dense_matrix.shape)
display(dense_matrix.ndim)
count_greater_than_1 = np.sum(dense_matrix >= 1.0)
print("大于等于1的单元格个数:", count_greater_than_1)

array([[0., 0., 0., ..., 0., 2., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

(21954, 25269)

2

大于等于1的单元格个数: 11039225
CPU times: total: 4.12 s
Wall time: 5.06 s


二维数组→df

In [51]:
%%time
dense_matrix_df = pd.DataFrame(dense_matrix, index=test_data.obs.index, columns=test_data.var.index)
display(dense_matrix_df)
# 使用条件筛选和np.sum()统计大于等于1的单元格个数#11039225
count_greater_than_1_2 = (dense_matrix_df >= 1).sum().sum()
print("大于等于1的单元格个数:", count_greater_than_1_2)#验证非零个数
df_test = dense_matrix_df.copy()#备份，准备运算
print(len(set(df_test.index.tolist())))#看总体数据UMIs是否重复→ 无重复
print(len(set(df_test.columns.tolist())))#看总体数据genes是否重复→ 无重复
print(df_test.max().max())
print(df_test.min().min())

,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A4GALT,AA415398,AA465934,AA467197,...,hsa-mir-3181,hsa-mir-335,hsa-mir-4259,hsa-mir-4537,hsa-mir-4538,hsa-mir-490,hsa-mir-5195,hsa-mir-6080,hsa-mir-8072,snoU2-30
AGAGCGAAGATCTGCT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
ACATCAGTCTGACCTC,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
TTCTCCTGTCCTGCTT,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
GAAACTCTCATGTAGC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
CAGTCCTGTCTAGCCG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTATGCTCTTGTACT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ATAACGCCAAGCTGGA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ATTACTCAGTGGACGT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CGTGTCTCAAGCGTAG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


大于等于1的单元格个数: 11039225
21954
25269
3053.0
0.0
CPU times: total: 15.1 s
Wall time: 18 s


读取训练集数据

In [52]:
train_data = pd.read_csv('train/training.csv',header = None, names = ['UMIs','result'])
df_train = train_data.copy()#备份
display(df_train)#y_train
count_nonzero = (df_train['result'] != 0).sum()
print(count_nonzero)
#print(len(set(df_train['UMIs'])))

,UMIs,result
0,GCTCTGTCAATGGATA,1
1,GATGAGGGTACGAAAT,0
2,AGGCCACGTACAGCAG,0
3,ACGCCGAGTCACACGC,0
4,TCTTCGGAGGCTAGCA,0
...,...,...
995,ACTGCTCCACTCGACG,0
996,TGGTTAGGTAAACGCG,0
997,TTCTCAATCAGTACGT,1
998,TGAGCCGGTCTCTTAT,0


145


看下training 的UMIs是否存在于df_test当中

In [53]:
display(df_test.loc[list(df_train['UMIs'])])
df_train2 = df_test.loc[list(df_train['UMIs'])].copy()#X_train
#df_train2.to_csv('processing/X_train.csv',encoding = 'utf-8')

,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A4GALT,AA415398,AA465934,AA467197,...,hsa-mir-3181,hsa-mir-335,hsa-mir-4259,hsa-mir-4537,hsa-mir-4538,hsa-mir-490,hsa-mir-5195,hsa-mir-6080,hsa-mir-8072,snoU2-30
GCTCTGTCAATGGATA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GATGAGGGTACGAAAT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AGGCCACGTACAGCAG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ACGCCGAGTCACACGC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TCTTCGGAGGCTAGCA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACTGCTCCACTCGACG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TGGTTAGGTAAACGCG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TTCTCAATCAGTACGT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TGAGCCGGTCTCTTAT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
importlib.reload(mf)
row_counts = df_train2.apply(lambda row: (row >= 1).sum(), axis=1)
# 打印每一行中大于等于1的数的数量
print(row_counts)
print(type(row_counts))
df_row_counts = mf.series_to_df(row_counts,'count')
display(df_row_counts)

GCTCTGTCAATGGATA    522
GATGAGGGTACGAAAT    593
AGGCCACGTACAGCAG    334
ACGCCGAGTCACACGC    219
TCTTCGGAGGCTAGCA     42
                   ... 
ACTGCTCCACTCGACG    424
TGGTTAGGTAAACGCG    492
TTCTCAATCAGTACGT    509
TGAGCCGGTCTCTTAT    317
AGGTCCGTCCATGCTC     64
Length: 1000, dtype: int64
<class 'pandas.core.series.Series'>


,index,count
357,AAGTCTGAGCGTCTAT,7459
469,CATGACATCTATCGCC,6991
309,AGATTGCGTTTGGGCC,6783
129,GTCCTCATCCTTGGTC,6164
376,AAGTCTGGTCGGGTCT,5957
...,...,...
710,GCTTGAACATGCCACG,34
178,GGTGCGTCATCTGGTA,34
195,CTCGAGGGTGAAGGCT,33
802,GCGCAGTGTTGTGGAG,28


In [55]:
display(df_row_counts[df_row_counts['count']>=1])

,index,count
357,AAGTCTGAGCGTCTAT,7459
469,CATGACATCTATCGCC,6991
309,AGATTGCGTTTGGGCC,6783
129,GTCCTCATCCTTGGTC,6164
376,AAGTCTGGTCGGGTCT,5957
...,...,...
710,GCTTGAACATGCCACG,34
178,GGTGCGTCATCTGGTA,34
195,CTCGAGGGTGAAGGCT,33
802,GCGCAGTGTTGTGGAG,28


In [56]:
print('finish2')

finish2


df_test 归一化 → df_test = X_train + X_test
 需要用到原始 test_data 格式

In [57]:
%%time
test_norm = sc.pp.normalize_total(test_data, exclude_highly_expressed = False , inplace=False)['X']
display(test_norm)
display(type(test_norm))
dense_test_norm = test_norm.toarray()#非零稀疏索引转为二维数组
display(dense_test_norm)
df_test_norm = pd.DataFrame(dense_test_norm, index=test_data.obs.index, columns=test_data.var.index).copy()
display(df_test_norm)

<21954x25269 sparse matrix of type '<class 'numpy.float64'>'
	with 11039225 stored elements in Compressed Sparse Column format>

scipy.sparse._csc.csc_matrix

array([[0.        , 0.        , 0.        , ..., 0.        , 0.01507565,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.0067953 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00832283,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A4GALT,AA415398,AA465934,AA467197,...,hsa-mir-3181,hsa-mir-335,hsa-mir-4259,hsa-mir-4537,hsa-mir-4538,hsa-mir-490,hsa-mir-5195,hsa-mir-6080,hsa-mir-8072,snoU2-30
AGAGCGAAGATCTGCT,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.015076,0.0
ACATCAGTCTGACCTC,0.0,0.0,0.0,0.0,0.006795,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.006795,0.0
TTCTCCTGTCCTGCTT,0.0,0.0,0.0,0.0,0.008323,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.008323,0.0
GAAACTCTCATGTAGC,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.016401,0.0
CAGTCCTGTCTAGCCG,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTATGCTCTTGTACT,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
ATAACGCCAAGCTGGA,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
ATTACTCAGTGGACGT,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
CGTGTCTCAAGCGTAG,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


CPU times: total: 19.7 s
Wall time: 26.1 s


PCA降维，降维到50,details未知

In [58]:
%%time
from scipy.sparse import csc_matrix
importlib.reload(mf)
pca_test = sc.pp.pca(anndata.AnnData(X = test_norm), n_comps= 50, zero_center=True, copy = True, svd_solver='auto', random_state=42)
display(pca_test.obsm['X_pca'])#※降维后50个维度的值（归一化后）
display(pca_test.obsm['X_pca'].shape)
display(type(pca_test.obsm['X_pca']))
# display(pca_test.varm['PCs'])#降维后与原维度的关系
# display(pca_test.varm['PCs'].shape)
# display(pca_test.uns['pca']['variance_ratio'])#残差参数
# display(pca_test.uns['pca']['variance_ratio'].shape)
# display(pca_test.uns['pca']['variance'])#残差原值
# display(pca_test.uns['pca']['variance'].shape)
display(type(pca_test))
print(pca_test.shape)#→copy=true的缘故

array([[ 2.7636597 , -1.3241842 , 14.388162  , ...,  0.04198899,
        -0.15336873, -0.35298958],
       [ 6.394659  ,  0.8012573 , 14.412637  , ...,  0.3020041 ,
        -1.1140212 ,  0.6212355 ],
       [ 5.034913  ,  0.3228025 , 15.289363  , ...,  0.29057726,
        -0.42864457,  0.5044053 ],
       ...,
       [ 4.8657236 , -0.7317507 , 12.288434  , ..., -2.513426  ,
         1.2307168 ,  6.306153  ],
       [-7.522014  ,  1.0591507 ,  6.462868  , ...,  7.9399366 ,
         6.336762  , -5.4263906 ],
       [-0.29170963,  9.750389  ,  2.7973833 , ..., -0.44656622,
         0.49417102,  5.702416  ]], dtype=float32)

(21954, 50)

numpy.ndarray

anndata._core.anndata.AnnData

(21954, 25269)
CPU times: total: 15.5 s
Wall time: 16.2 s


纳入模型数据整理
要求是归一化后的数据


In [59]:
%%time
#X_train --df_train2
#思路，要在training.csv的UMIs找出其在test_data的行索引，后在pca_test中提取
matching_indices = np.where(df_test.index.isin(df_train['UMIs']))[0]#array
X_train = pca_test.obsm['X_pca'][matching_indices]
X_train_ori = test_norm[matching_indices]
#X_test --(df_test - df_train2) ,X_trian的补集
X_test = pca_test.obsm['X_pca'][[i for i in range(len(df_test.index)) if i not in matching_indices]]
X_test_ori = test_norm[[i for i in range(len(df_test.index)) if i not in matching_indices]]
print(X_train.shape)
print(X_test.shape)
#y_train training.csv--y_train#需要一维数组
y_train = df_train['result'].values
print(y_train.shape)
#y_test prediction.csv--y_test
y_test = pd.read_csv("prediction/prediction.csv",header=None).copy()
y_test.columns = ['UMIs', 'result']
display(y_test)
y_test = y_test['result'].values
print(y_test.shape)

(1000, 50)
(20954, 50)
(1000,)


,UMIs,result
0,AGAGCGAAGATCTGCT,0
1,ACATCAGTCTGACCTC,0
2,TTCTCCTGTCCTGCTT,0
3,GAAACTCTCATGTAGC,0
4,CAGTCCTGTCTAGCCG,0
...,...,...
20949,TTTATGCTCTTGTACT,0
20950,ATAACGCCAAGCTGGA,0
20951,ATTACTCAGTGGACGT,0
20952,CGTGTCTCAAGCGTAG,0


(20954,)
CPU times: total: 781 ms
Wall time: 850 ms


跑模型，随机森林为例 文件 myRF

In [64]:
%%time
import my_RF as mr
importlib.reload(mr)
mcc0 =  mr.rf(X=None,y=None,mode='test',X_train=X_train, X_test=None, y_train=y_train, y_test=None,pattern = 0,fund = 20)#0或1 ，看那个结果好
_,mcc1 = mr.rf(X=None,y=None,mode='test',X_train=X_train, X_test=None, y_train=y_train, y_test=None,pattern = 1,fund = 20)#0或1 ，看那个结果好
mcc3 = mr.gbdt(X=None,y=None,mode='test',X_train=X_train, X_test=None, y_train=y_train, y_test=None,pattern = 0,fund = 20)
mcc4 = mr.xgboost(X=None,y=None,mode='test',X_train=X_train, X_test=None, y_train=y_train, y_test=None,pattern = 0,fund = 20)
print(mcc0)
print(mcc1)
print(mcc3)
print(mcc4)

D:\Hackathon-Summer-2023\my_RF.py:58: RuntimeWarning: invalid value encountered in divide
  f1_scores = 2 * (precision * recall) / (precision + recall)


[15:42:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0fdc6d574b9c0d168-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "max_samples" } are not used.

0.0
-0.10264482299902934
-0.02555352081916642
-0.03622927869271666
CPU times: total: 1min 45s
Wall time: 1min 51s


In [61]:
# %%time
# importlib.reload(mr)
# mcc0 =  mr.rf(X=None,y=None,mode='train',X_train=X_train_ori, X_test=X_test_ori, y_train=y_train, y_test=y_test,pattern = 0)#0或1 ，看那个结果好
# _,mcc1 = mr.rf(X=None,y=None,mode='train',X_train=X_train_ori, X_test=X_test_ori, y_train=y_train, y_test=y_test,pattern = 1)#0或1 ，看那个结果好
# mcc3 = mr.gbdt(X=None,y=None,mode='train',X_train=X_train_ori, X_test=X_test_ori, y_train=y_train, y_test=y_test,pattern = 0)
# print(mcc0)
# print(mcc1)
# print(mcc3)

循环筛选

In [62]:
%%time
import random
def loop_screen():
    r_list = []
    fund_list = []
    n_list = []
    importlib.reload(mf)
    import my_RF as mr
    importlib.reload(mr)
    n = 1
    flag = True
    while flag:
        random_integer = random.randint(10, 40)
        fund = random_integer
        pca_test = sc.pp.pca(anndata.AnnData(X = test_norm), n_comps= n, zero_center=True, copy = True, svd_solver='auto', random_state= 42)
        X_train = pca_test.obsm['X_pca'][matching_indices]
        X_test = pca_test.obsm['X_pca'][[i for i in range(len(df_test.index)) if i not in matching_indices]]
        mcc1 =  mr.rf(X=None,y=None,mode='train',X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,pattern = 0, fund = fund)#0或1 ，看那个结果好
        mcc2 = mr.gbdt(X=None,y=None,mode='train',X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,pattern = 0, fund = fund)
        mcc3 = mr.xgboost(X=None,y=None,mode='train',X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,pattern = 0, fund = fund)
        final_mcc = max([mcc1,mcc2,mcc3])
        final_idx = ([mcc1,mcc2,mcc3].index(final_mcc))
        if final_idx ==0:
            r = mcc1
        elif final_idx ==1:
            r = mcc2
        else:
            r = mcc3
        if r > 0.01:
            r_list.append(r)
            fund_list.append(fund)
            n_list.append(n)
            r_df  = pd.DataFrame({'r':r_list,'fund':fund_list,'n':n_list})
            print(r_df)
            return r_df
        n += 1
    print('finish')

CPU times: total: 0 ns
Wall time: 999 µs


In [63]:
%%time
r_df = loop_screen()
display(r_df)

ValueError: y contains 1 class after sample_weight trimmed classes with zero weights, while a minimum of 2 classes are required.